In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
df = pd.read_csv("filtered_list.csv")
df

,Track URI,Track Name,Artist Name(s),Duration (ms),Popularity,Explicit,User,playlist year,release_year
0,spotify:track:2D2lazsae9o1UoVPUAdxyT,21,Gracie Abrams,185172,31,False,Emily,2023,2020
1,spotify:track:2RvLlxea9woaqb2ZLgAmv7,emma,Casper,195066,40,False,Emily,2023,2023
2,spotify:track:53dtP2iUMvaF28JZcHnFuU,logical,Olivia Rodrigo,231907,71,False,Emily,2023,2023
3,spotify:track:401qSkk4KOeBVld2aiCAoK,Lovers to Strangers,Chance Peña,174086,52,True,Emily,2023,2023
4,spotify:track:2hkcv7xQJgmLvCY3XSixjG,Dussmann,Betterov,217551,45,False,Emily,2023,2022
...,...,...,...,...,...,...,...,...,...
1195,spotify:track:2jdAk8ATWIL3dwT47XpRfu,Slow Dancing in a Burning Room,John Mayer,241946,78,False,William,2025,2006
1196,spotify:track:4QlzkaRHtU8gAdwqjWmO8n,Friday I'm In Love,The Cure,215160,79,False,William,2025,1992
1197,spotify:track:3wlNmUeoHnM1jcOcXxC6qI,Rivendell,Howard Shore,206026,56,False,William,2025,2001
1198,spotify:track:3p428gAKm9dBoMjBVOIvVu,I Can't Hear It Now (from the series Arcane Le...,Arcane;Freya Ridings,161040,64,False,William,2025,2024


In [2]:
# add duration in minutes (float) and as mm:ss
df['Duration_min'] = (df['Duration (ms)'] / 60000).round(2)

def ms_to_mmss(ms):
    sec = int(ms // 1000)
    m = sec // 60
    s = sec % 60
    return f"{m}:{s:02d}"

df['Duration_mmss'] = df['Duration (ms)'].apply(ms_to_mmss)

# display a preview
df[['Track Name', 'Artist Name(s)', 'Duration (ms)', 'Duration_min', 'Duration_mmss']].head()

,Track Name,Artist Name(s),Duration (ms),Duration_min,Duration_mmss
0,21,Gracie Abrams,185172,3.09,3:05
1,emma,Casper,195066,3.25,3:15
2,logical,Olivia Rodrigo,231907,3.87,3:51
3,Lovers to Strangers,Chance Peña,174086,2.90,2:54
4,Dussmann,Betterov,217551,3.63,3:37


In [3]:
# Numeric columns
numeric_cols = df.select_dtypes(include='number').columns.tolist()

# Boolean columns (True / False)
bool_cols = df.select_dtypes(include='bool').columns.tolist()

# Combined selector options
stat_cols = numeric_cols + bool_cols

stat_cols


['Duration (ms)',
 'Popularity',
 'playlist year',
 'release_year',
 'Duration_min',
 'Explicit']

In [4]:
output = widgets.Output()

def user_stats_plot(column):
    output.clear_output(wait=True)

    with output:
        # Boolean column handling
        if column in bool_cols:
            stats = (
                df
                .assign(_bool_as_int=df[column].astype(int))
                .groupby('User')['_bool_as_int']
                .agg(
                    mean='mean',   # proportion explicit
                    sum='sum',     # count explicit
                    min='min',
                    max='max'
                )
            )

            display(stats)
        # Numeric column handling
        else:
            stats = (
                df
                .groupby('User')[column]
                .agg(['mean', 'median', 'min', 'max'])
            )

            display(stats)
        plt.tight_layout()
        plt.show()


In [5]:
column_selector = widgets.Dropdown(
    options=stat_cols,
    description='Column:'
)

column_selector.observe(
    lambda change: user_stats_plot(change['new']),
    names='value'
)

display(column_selector, output)

# Initial render
user_stats_plot(column_selector.value)


Dropdown(description='Column:', options=('Duration (ms)', 'Popularity', 'playlist year', 'release_year', 'Dura…

Output()

In [24]:

import webbrowser
songs_per_user = 10

selected_songs = (
    df
    .groupby('User', group_keys=False)
    .apply(lambda x: x.sample(n=songs_per_user, random_state=None))
    .reset_index(drop=True)
)
len(selected_songs)
game_playlist = selected_songs.sample(frac=1).reset_index(drop=True)


current_index = 0

def play_next_song():
    global current_index

    if current_index >= len(game_playlist):
        print("🎉 Game over — no more songs!")
        return

    row = game_playlist.iloc[current_index]

    track_uri = row['Track URI']
    track_name = row['Track Name']
    artist = row['Artist Name(s)']
    selected_user = row['User']
    release_year = row['release_year']

    # Cross-check with original df
    matches = df[
        (df['Track Name'] == track_name) &
        (df['Artist Name(s)'] == artist)
    ]

    users_with_track = sorted(matches['User'].unique().tolist())
    playlist_years = sorted(matches['playlist year'].dropna().unique().tolist())

    print(f"🎵 Now playing ({current_index + 1}/{len(game_playlist)})")
    print(f"Track: {track_name}")
    print(f"Artist: {artist}")
    print(f"Release year: {release_year}")
    print(f"Playlist year(s): {playlist_years}")
    print(f"Appears in playlists of: {users_with_track}")
    print(f"Selected from: {selected_user}")

    webbrowser.open(track_uri)

    current_index += 1



C:\Users\willi\AppData\Local\Temp\ipykernel_7788\1944218852.py:7: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=songs_per_user, random_state=None))


In [25]:
game_playlist

,Track URI,Track Name,Artist Name(s),Duration (ms),Popularity,Explicit,User,playlist year,release_year,Duration_min,Duration_mmss
0,spotify:track:2mruPq4Gcx8tPpkQawsrbl,DXMINXS,TWISTED;Scarlxrd,176913,36,True,Keke,2023,2022,2.95,2:56
1,spotify:track:5ivmcr2ey37sIiZGlKu9IA,Epilog?,Eden Bleak,187889,14,False,Emily,2023,2022,3.13,3:07
2,spotify:track:4lZ7OpzFPYJIWVgfuWU0zv,Dies & Das,01099;Gustav;Zachi,140466,58,False,Keke,2024,2022,2.34,2:20
3,spotify:track:4or7DZfr1xrfurpJhleZa6,Geister,Eden Bleak,161298,3,False,Emily,2023,2022,2.69,2:41
4,spotify:track:5nq0wConoTHndauu719R2I,Ніч яка місячна (feat. Tember Blanche),mine;Tember Blanche,221007,15,False,William,2023,2023,3.68,3:41
5,spotify:track:7tCHpjktA50ihtkLz6bAnn,Ain't No Rest For The Wicked - Original Version,Cage The Elephant,175493,66,False,Lewis,2023,2008,2.92,2:55
6,spotify:track:61nkMojCSJuXspIOWmo0vE,BLXXDY NXSE (feat. Scarlxrd & nascar aloe),Jasiah;Scarlxrd;nascar aloe,187317,36,True,Keke,2023,2023,3.12,3:07
7,spotify:track:1nuYw5614RF5tm7Byrigw5,Luktelk,Silvester Belt,161142,0,False,William,2024,2024,2.69,2:41
8,spotify:track:4kbkYbhWNiSJraySknB4hD,Baller,Abor & Tynna,159493,66,False,Emily,2025,2025,2.66,2:39
9,spotify:track:5E2JcyolDstvqZ0PNpT3pS,SUVs,Luciano,162173,56,False,Keke,2024,2022,2.70,2:42


In [65]:
play_next_song()


🎵 Now playing (40/40)
Track: CARNIVAL
Artist: ¥$;Kanye West;Ty Dolla $ign;Rich The Kid;Playboi Carti
Release year: 2024
Playlist year(s): [2024]
Appears in playlists of: ['Keke']
Selected from: Keke
